In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_probability as tfp
from sklearn.model_selection import train_test_split

tfd = tfp.distributions
tfb = tfp.bijectors
tfpl = tfp.layers

In [ ]:
items_features = pd.read_csv('../dataset/processed_data/simplified_features_and_categories_30.csv')
items_features.head()

,item_id,feature_idx
0,19021,0
1,19021,22
2,19021,15
3,19021,446
4,19021,1


In [ ]:
items_features['item_id'].max()

23691

In [ ]:
dataset = np.zeros(
    (items_features['item_id'].max()+1, items_features['feature_idx'].max()+1)
    )
dataset[(items_features['item_id'], items_features['feature_idx'])] = 1

dataset
train_dataset=dataset[1:,:]

In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(train_dataset,test_size=0.2,random_state=941116)

In [ ]:
dataset.shape

(23692, 636)

In [ ]:
train.shape

(18952, 636)

In [ ]:
test.shape

(4739, 636)

# Autoencoder

In [ ]:
latent_dimension = 64
starting_dimension = 512

inputs = layers.Input(items_features['feature_idx'].max()+1)

x = inputs 
dimension = starting_dimension

while(dimension != latent_dimension):
    #x = layers.BatchNormalization()(x)
    x = layers.Dense(units=dimension, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
    x = layers.Dropout(0.1)(x)

    dimension = dimension//2

x = layers.Dense(units=dimension,activation="tanh",activity_regularizer=tf.keras.regularizers.L1L2(l1=5e-5,l2=4e-4))(x)
outputs = x
encoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='encoder')

In [ ]:
max_width = 1024
ending_dimension = items_features['feature_idx'].max()+1

inputs = layers.Input(latent_dimension)

x = inputs 
dimension = latent_dimension

x = layers.Dense(units=256, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(units=512, activation='gelu',activity_regularizer=tf.keras.regularizers.L1L2(l1=1e-5,l2=1e-5))(x)
x = layers.Dropout(0.2)(x)
    

x = layers.Dense(units=ending_dimension,activation="sigmoid")(x)
outputs = x

decoder = tf.keras.Model(inputs=inputs, outputs=outputs, name='decoder')

#decoder.summary()

In [ ]:
ae = tf.keras.Model(inputs=encoder.inputs, outputs=decoder(encoder.outputs), name='auto_encoder')
#ae.summary()

In [ ]:
def custom_loss(x_true,x_pred):
    return tf.keras.losses.cosine_similarity(x_true,x_pred)+tf.keras.losses.mean_squared_error(x_true,x_pred)#

In [ ]:
ae.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.006), 
    loss=custom_loss,
    metrics=[tf.keras.metrics.CosineSimilarity(),tf.keras.metrics.MeanAbsoluteError()]
    )

#Training

In [ ]:
ae.fit(
    train,
    train, 
    batch_size=64,
    validation_data=(test,test),
    epochs=200,
    shuffle=True,
    callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='cosine_similarity',
                patience=10,
                restore_best_weights=True,
                mode="max"
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_cosine_similarity',
                factor=0.7,
                patience=2,
                min_lr=5e-5,
                mode="max",
                min_delta=0.0001,
            )
        ]
    )

Epoch 1/200
297/297 [==============================] - 8s 21ms/step - loss: -0.7541 - cosine_similarity: 0.7911 - mean_absolute_error: 0.0499 - val_loss: -0.8476 - val_cosine_similarity: 0.8716 - val_mean_absolute_error: 0.0317 - lr: 0.0060
Epoch 2/200
297/297 [==============================] - 6s 20ms/step - loss: -0.8534 - cosine_similarity: 0.8763 - mean_absolute_error: 0.0302 - val_loss: -0.8736 - val_cosine_similarity: 0.8938 - val_mean_absolute_error: 0.0263 - lr: 0.0060
Epoch 3/200
297/297 [==============================] - 6s 19ms/step - loss: -0.8728 - cosine_similarity: 0.8931 - mean_absolute_error: 0.0265 - val_loss: -0.8909 - val_cosine_similarity: 0.9095 - val_mean_absolute_error: 0.0222 - lr: 0.0060
Epoch 4/200
297/297 [==============================] - 6s 19ms/step - loss: -0.8858 - cosine_similarity: 0.9045 - mean_absolute_error: 0.0239 - val_loss: -0.8989 - val_cosine_similarity: 0.9161 - val_mean_absolute_error: 0.0211 - lr: 0.0060
Epoch 5/200
297/297 [===============

In [ ]:
temp=encoder(dataset)

In [ ]:
temp.shape

TensorShape([23692, 64])

In [ ]:
temp

In [ ]:
tf.reduce_sum(temp,axis=1).numpy().max(),tf.reduce_sum(temp,axis=1).numpy().min(),tf.reduce_sum(temp,axis=1).numpy().std()

5.268532

In [ ]:
np.save('../dataset/processed_data/compressed_features', encoder(dataset))